In [ ]:
pip install transformers datasets accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
import os
from torch.utils import data
import torch
import csv
from sklearn.preprocessing import LabelEncoder
import pickle
import json
from pprint import pprint

# Training the model : relation(none/support+attack)




In the training dataset and validation dataset,none has been set to 0, and support and attack have been set to 1

## Split each item in the dataset into individual components

The dataset is divided into three parts Train dataset (used for training)、Validation dataset (used to test training progress after each training session)、Test dataset (used for final evaluation)

In [ ]:
with open('train_social_relation.json','r',encoding='utf-8') as file:
    data = json.load(file)
    train_sen1 = []
    train_sen2 = []
    trainlabel = []
    for line in data:
      train_sen1.append(line[0])
      train_sen2.append(line[1])
      trainlabel.append(line[2])

In [ ]:
with open('test_social_relation.json','r',encoding='utf-8') as file:
    data = json.load(file)
    test_sen1 = []
    test_sen2 = []
    testlabel = []
    for line in data:
      test_sen1.append(line[0])
      test_sen2.append(line[1])
      testlabel.append(line[2])

In [ ]:
with open('dev_social_relation.json','r',encoding='utf-8') as file:
    data = json.load(file)
    eval_sen1 = []
    eval_sen2 = []
    evallabel = []
    for line in data:
      eval_sen1.append(line[0])
      eval_sen2.append(line[1])
      evallabel.append(line[2])

## Choosing and using a tokenizer

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

In [ ]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [ ]:
train_encodings = tokenizer(train_sen1, train_sen2, truncation=True, padding=True)
print(train_encodings)
eval_encodings = tokenizer(eval_sen1, eval_sen2, truncation=True, padding=True)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
print(tokenizer.decode(train_encodings['input_ids'][0]))
print(train_encodings['input_ids'][0])
print("token_type_ids\n", train_encodings['token_type_ids'][0])
print("attention_mask\n", train_encodings['attention_mask'][0])

print(len(train_encodings['input_ids'][0]))
print(len(train_encodings['token_type_ids'][0]))

[CLS] 來 了 來 了 上 沖 下 洗 先 拉 後 跌 先 洗 個 幾 輪 意 志 不 堅 的 賭 徒 就 被 洗 出 場 了 真 爽 [SEP] 哈 哈 不 是 都 説 了 一 支 押 賭 不 要 超 過 一 個 月 薪 水 嗎 ？ 一 個 月 薪 水 變 壁 紙 又 不 痛 不 癢 你 要 賭 身 家 我 也 沒 辦 法 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

## Adding labels (answers)

In [ ]:
def add_targets(encodings,label):
    encodings.update({'label':label})
add_targets(train_encodings,trainlabel)
add_targets(eval_encodings,evallabel)


In [ ]:
print(train_encodings.keys())
print(tokenizer.decode(train_encodings['label']))

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'label'])
[PAD] [unused1] [unused1] [PAD] [unused1] [unused1] [PAD] [unused1] [unused1] [PAD] [unused1] [PAD] [unused1] [unused1] [unused1] [unused1] [unused1] [PAD] [PAD] [unused1] [unused1] [unused1] [unused1] [unused1] [unused1] [PAD] [unused1] [unused1] [unused1] [unused1] [PAD] [PAD] [PAD] [unused1] [unused1] [unused1] [unused1] [PAD] [unused1] [unused1] [unused1] [unused1] [PAD] [PAD] [unused1] [unused1] [PAD] [PAD] [unused1] [PAD] [unused1] [unused1] [unused1] [unused1] [PAD] [unused1] [unused1] [PAD] [PAD] [PAD] [unused1] [PAD] [PAD] [unused1] [unused1] [PAD] [unused1] [PAD] [unused1] [PAD] [PAD] [PAD] [PAD] [PAD] [unused1] [PAD] [unused1] [unused1] [unused1] [unused1] [unused1] [PAD] [unused1] [unused1] [PAD] [unused1] [unused1] [unused1] [PAD] [PAD] [unused1] [unused1] [unused1] [unused1] [unused1] [unused1] [unused1] [unused1] [unused1] [unused1] [unused1] [PAD] [PAD] [PAD] [unused1] [PAD] [unused1] [PAD] [unused1] 

## Defining a dataset and converting it into tensor format


In [ ]:
from torch.utils import data
import torch

class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
    return {key: torch.tensor(eval[idx]) for key, eval in self.encodings.items()}

  def __len__(self):
    return len(self.encodings.input_ids)

In [ ]:
train_dataset = Dataset(train_encodings)
eval_dataset = Dataset(eval_encodings)

In [ ]:
train_dataset[0]

{'input_ids': tensor([ 101,  889,  749,  889,  749,  677, 3762,  678, 3819, 1044, 2861, 2527,
         6649, 1044, 3819,  943, 2407, 6743, 2692, 2562,  679, 1830, 4638, 6551,
         2530, 2218, 6158, 3819, 1139, 1842,  749, 4696, 4272,  102, 1506, 1506,
          679, 3221, 6963, 6304,  749,  671, 3118, 2852, 6551,  679, 6206, 6631,
         6882,  671,  943, 3299, 5959, 3717, 1621, 8043,  671,  943, 3299, 5959,
         3717, 6365, 1880, 5158, 1348,  679, 4578,  679, 4623,  872, 6206, 6551,
         6716, 2157, 2769,  738, 3760, 6794, 3791,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0, 

## Loading the model architecture

In [ ]:
from transformers import BertConfig, BertForSequenceClassification
config = BertConfig.from_pretrained('bert-base-chinese', num_labels=2)  #num_labels : Setting the number of classes
model = BertForSequenceClassification.from_pretrained("bert-base-chinese",config=config)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## Training the model

In [ ]:
import logging
import datasets
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
import math

import transformers
from accelerate import Accelerator
from transformers import (
    AdamW,
    AutoConfig,
    default_data_collator,
    get_scheduler
)

train_batch_size = 8      # Setting training batch size
eval_batch_size = 8      # Setting eval batch size
num_train_epochs = 5      # Setting epoch

In [ ]:
data_collator = default_data_collator
train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=data_collator, batch_size=train_batch_size)
eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=eval_batch_size)

In [ ]:
learning_rate=3e-5          # Setting learning_rate
gradient_accumulation_steps = 1   # Setting the number of steps before performing backpropagation

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
max_train_steps = num_train_epochs * num_update_steps_per_epoch
print('max_train_steps', max_train_steps)

# scheduler
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=max_train_steps,
)

max_train_steps 6210


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

eval_dataloader
metric = load_metric("accuracy")

<ipython-input-31-570f2d83a7b4>:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [ ]:
output_bat=[]
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state)
output_dir = '/content'  # your folder

total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {max_train_steps}")


completed_steps = 0
best_epoch = {"epoch": 0, "acc": 0 }

for epoch in trange(num_train_epochs, desc="Epoch"):#trange is a way to print a progress bar
  model.train()
  for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
    outputs = model(**batch)
    #loss = outputs
    loss = outputs.loss
    #loss = loss / gradient_accumulation_steps
    accelerator.backward(loss)
    #if step % gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1: Removed the 'if' statement
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    completed_steps += 1

    if step % 50 == 0:
      print({'epoch': epoch, 'step': step, 'loss': loss.item()})

    if completed_steps >= max_train_steps:
      break

  logger.info("***** Running eval *****")
  model.eval()
  for step, batch in enumerate(tqdm(eval_dataloader, desc="eval Iteration")):
    outputs = model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    metric.add_batch(
        predictions=accelerator.gather(predictions),
        references=accelerator.gather(batch["labels"]),
    )

  eval_metric = metric.compute()
  logger.info(f"epoch {epoch}: {eval_metric}")
  output_bat.append({"epoch": epoch, "acc": eval_metric['accuracy']})
  if eval_metric['accuracy'] > best_epoch['acc']:
    best_epoch.update({"epoch": epoch, "acc": eval_metric['accuracy']})

  if output_dir is not None:
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir + '/' + 'epoch_' + str(epoch), save_function=accelerator.save)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1242 [00:00<?, ?it/s]

{'epoch': 0, 'step': 0, 'loss': 0.6567797660827637}


## Bert epoch

In [ ]:
print(best_epoch)

Save "/content/epoch_3/config.json" to ***/content/drive/MyDrive/Colab Notebooks/relation_config.json***

Save "/content/epoch_3/pytorch_model.bin" to ***/content/drive/MyDrive/Colab Notebooks/relation_pytorch_model.bin***

# Training the model : relation(support、attack)

In the dataset,delete none,and support has been set to 0,attack have been set to 1

In [ ]:
with open('train_social_att_sup.json','r',encoding='utf-8') as file:
    data = json.load(file)
    train_sen1 = []
    train_sen2 = []
    trainlabel = []
    for line in data:
      train_sen1.append(line[0])
      train_sen2.append(line[1])
      trainlabel.append(line[2])

In [ ]:
with open('dev_social_att_sup.json','r',encoding='utf-8') as file:
    data = json.load(file)
    eval_sen1 = []
    eval_sen2 = []
    evallabel = []
    for line in data:
      eval_sen1.append(line[0])
      eval_sen2.append(line[1])
      evallabel.append(line[2])

In [ ]:
train_encodings = tokenizer(train_sen1, train_sen2, truncation=True, padding=True)
eval_encodings = tokenizer(eval_sen1, eval_sen2, truncation=True, padding=True)

In [ ]:
train_encodings.keys()

In [ ]:
#Adding labels (answers)
add_targets(train_encodings,trainlabel)
add_targets(eval_encodings,evallabel)

In [ ]:
train_dataset = Dataset(train_encodings)
eval_dataset = Dataset(eval_encodings)

##Loading the model architecture

In [ ]:
from transformers import BertConfig, BertForSequenceClassification
config = BertConfig.from_pretrained('bert-base-chinese', num_labels=2)  #num_labels 設定類別數
model = BertForSequenceClassification.from_pretrained("bert-base-chinese",config=config)

In [ ]:
print(model)

##training

In [ ]:
import logging
import datasets
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
import math

import transformers
from accelerate import Accelerator
from transformers import (
    AdamW,
    AutoConfig,
    default_data_collator,
    get_scheduler
)

train_batch_size = 8
eval_batch_size = 8
num_train_epochs = 5

In [ ]:
data_collator = default_data_collator
train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=data_collator, batch_size=train_batch_size)
eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=eval_batch_size)

In [ ]:
learning_rate=3e-5
gradient_accumulation_steps = 1

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
max_train_steps = num_train_epochs * num_update_steps_per_epoch
print('max_train_steps', max_train_steps)

# scheduler
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=max_train_steps,
)

In [ ]:

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

eval_dataloader
metric = load_metric("accuracy")

In [ ]:
output_bat=[]
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state)
output_dir = '/content'  # your folder

total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {max_train_steps}")


completed_steps = 0
best_epoch = {"epoch": 0, "acc": 0 }

for epoch in trange(num_train_epochs, desc="Epoch"):
  model.train()
  for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
    outputs = model(**batch)
    #loss = outputs
    loss = outputs.loss
    #loss = loss / gradient_accumulation_steps
    accelerator.backward(loss)
    #if step % gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1: Removed the 'if' statement
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    completed_steps += 1

    if step % 50 == 0:
      print({'epoch': epoch, 'step': step, 'loss': loss.item()})

    if completed_steps >= max_train_steps:
      break

  logger.info("***** Running eval *****")
  model.eval()
  for step, batch in enumerate(tqdm(eval_dataloader, desc="eval Iteration")):
    outputs = model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    metric.add_batch(
        predictions=accelerator.gather(predictions),
        references=accelerator.gather(batch["labels"]),
    )

  eval_metric = metric.compute()
  logger.info(f"epoch {epoch}: {eval_metric}")
  output_bat.append({"epoch": epoch, "acc": eval_metric['accuracy']})
  if eval_metric['accuracy'] > best_epoch['acc']:
    best_epoch.update({"epoch": epoch, "acc": eval_metric['accuracy']})

  if output_dir is not None:
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir + '/' + 'epoch_' + str(epoch), save_function=accelerator.save)

In [ ]:
print(best_epoch)

Save "/content/epoch_2/config.json" to ***/content/drive/MyDrive/Colab Notebooks/att_sup_config.json***

Save "/content/epoch_2/pytorch_model.bin" to ***/content/drive/MyDrive/Colab Notebooks/att_sup_pytorch_model.bin***

# Validation of training results on official dataset

In [ ]:
from transformers import BertTokenizerFast, BertConfig, BertForSequenceClassification, default_data_collator
from torch.utils.data import DataLoader
from accelerate import Accelerator
from tqdm.auto import tqdm

In [ ]:
config = BertConfig.from_pretrained("/content/drive/MyDrive/Colab Notebooks/relation_config.json")
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/relation_pytorch_model.bin", config = config).to(device)

In [ ]:
from datasets import list_datasets, load_datasefrom pprint import pprint

##Classify into none/support+attack first

In [ ]:
def mrpc_model(model, sen1, sen2):
  input_encodings = tokenizer([sen1], [sen2], padding='max_length', truncation=True)
  input_dataset = Dataset(input_encodings)
  #print(input_encodings)
  #print(input_dataset[0])
  data_collator = default_data_collator
  input_dataloader = DataLoader(input_dataset, collate_fn=data_collator, batch_size=1)

  accelerator = Accelerator()
  model, input_dataloader = accelerator.prepare(model, input_dataloader)

  for batch in input_dataloader:
    outputs = model(**batch)
    predicted = outputs.logits.argmax(dim=-1)
  return predicted

In [ ]:
data=[]
answer=[]
test=[]
att_sup_sen1=[]
att_sup_sen2=[]
att_sup_label=[]
import json
with open("/content/drive/MyDrive/Colab Notebooks/test_social.json","r",encoding='utf-8')as f:
    data=json.load(f)
print(len(data))
cnt=0
for i in data:
    sen1=i[0]
    sen2=i[1]
    label=i[2]
    predict = mrpc_model(model, sen1, sen2)

    if predict.item() == 1:
      att_sup_sen1.append(sen1)
      att_sup_sen2.append(sen2)
      att_sup_label.append(label)

    answer.append(predict.item())
    test.append(label)


## Classify into support and attack

In [ ]:
config2 = BertConfig.from_pretrained("/content/drive/MyDrive/Colab Notebooks/att_sup_config.json")
model2 = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/att_sup_pytorch_model.bin", config = config2).to(device)

In [ ]:
def mrpc_model2(model, sen1, sen2):
  input_encodings = tokenizer([sen1], [sen2], padding='max_length', truncation=True)
  input_dataset = Dataset(input_encodings)
  #print(input_encodings)
  #print(input_dataset[0])
  data_collator = default_data_collator
  input_dataloader = DataLoader(input_dataset, collate_fn=data_collator, batch_size=1)

  accelerator = Accelerator()
  model, input_dataloader = accelerator.prepare(model, input_dataloader)

  for batch in input_dataloader:
    outputs = model(**batch)
    predicted = outputs.logits.argmax(dim=-1)
  return predicted

In [ ]:
#Take the originally classified 1(support+attack) out, and then perform binary classification on them again
for i in range(len(answer)):
  if answer[i]==1:
    sen1=data[i][0]
    sen2=data[i][1]
    predict = mrpc_model2(model2, sen1, sen2)
    if predict.item()==0:
      answer[i]=2

## Final accuracy

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(answer,test))